In [ ]:

#from models.util import load_images
#from models.advanced_network import build_advanced_net
drive.mount('/content/drive')


# paths
train_dir = '/content/drive/MyDrive/Final Year Proj 21-22/Train/Binary'
val_dir = '/content/drive/MyDrive/Final Year Proj 21-22/Validation'
test_dir = '/content/drive/MyDrive/Final Year Proj 21-22/Test/Binary'
MODEL_DIR = '/content/drive/MyDrive/Final Year Proj 21-22/vit-mnist'

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization, Concatenate
from tensorflow.keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator
from keras import callbacks
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

#(train_images, train_labels), (test_images, test_labels) = keras.datasets.mnist.load_data()
#train_images = train_images.reshape(train_images.shape[0], 28, 28, 1)
#test_images = test_images.reshape(test_images.shape[0], 28, 28, 1)

def preprocess_images(imgs): # should work for both a single image and multiple images
    sample_img = imgs if len(imgs.shape) == 2 else imgs[0]
    assert sample_img.shape in [(28, 28, 1), (28, 28)], sample_img.shape # make sure images are 28x28 and single-channel (grayscale)
    return imgs / 255.0

NUM_EPOCHS = 50
BATCH_SIZE = 64
LEARN_RATE = 0.00001
TEST_RATIO = 0.2
SEED = 60
NUM_CLASSES = 2
IMAGE_SIZE=224



# augment images using image data generator
data_aug = ImageDataGenerator(rescale=1.0/255.0,
                                  rotation_range=40,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  fill_mode='nearest',
                         shear_range=0.15)
## ALEXNET
model = keras.Sequential()

model.add(Conv2D(32, kernel_size=(4, 4), activation='relu', input_shape=(224, 224, 1))) 
model.add(Conv2D(32, kernel_size=(4, 4), activation='relu'))
model.add(Conv2D(64, (4, 4), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (4, 4), activation='relu'))
model.add(Conv2D(64, (4, 4), activation='relu'))
model.add(Conv2D(128, (4, 4), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(256, activation='relu'))

model.add(Dense(128, activation='relu'))

model.add(Dropout(0.5))

model.add(Dense(2, activation='softmax'))

model.summary()

model.compile(optimizer=tf.optimizers.Adam(), 
              loss='binary_crossentropy',
              metrics=['accuracy'])




filepath = "modelCNN25thDec.h5"
#checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, 
#                            save_best_only=True, mode='max')

#reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=2, 
#                                   verbose=1, mode='max', min_lr=0.00001)
                              
                              
#callbacks_list = [checkpoint, reduce_lr]


history_main = model.fit(
                    data_aug.flow_from_directory(test_dir,target_size=(224,224), batch_size=BATCH_SIZE, shuffle=True,
                         color_mode='grayscale'),
                    epochs = 20, 
                    validation_data=data_aug.flow_from_directory(val_dir,target_size=(224,224), batch_size=BATCH_SIZE, shuffle=True,
                         color_mode='grayscale')#,callbacks=callbacks_list
                    )
# plot loss and accuracy history
plt.figure()
plt.plot(history_main.history['accuracy'])
plt.plot(history_main.history['val_accuracy'])
plt.title('Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'])
plt.savefig(MODEL_DIR + '/model_accuracy2.png')
plt.figure()
plt.plot(history_main.history['loss'])
plt.plot(history_main.history['val_loss'])
plt.title('Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'])
#plt.savefig(MODEL_DIR + '/model_loss2.png')

plt.show()

from tensorflow.keras.applications import ResNet50

input_ = tf.keras.Input(shape=(224, 224, 1))


img_conc=tf.keras.layers.Concatenate()([input_,input_,input_])


# Initialize the Pretrained Model
feature_extractor = ResNet50(weights='imagenet', 
                             input_tensor=img_conc,
                             include_top=False)

# Set this parameter to make sure it's not being trained
feature_extractor.trainable = False

# Set the input layer



x = Conv2D(32,(3,3),activation='relu' ,padding='same')(input_)
x = feature_extractor(input_, training=False)
# Set the feature extractor layer


# Set the pooling layer
x = tf.keras.layers.GlobalAveragePooling2D()(x)

# Set the final layer with sigmoid activation function
output_ = tf.keras.layers.Dense(2, activation='sigmoid')(x)

# Create the new model object
model = tf.keras.Model(input_, output_)

# Compile it
model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['accuracy'])

# Print The Summary of The Model
model.summary()

model.compile(optimizer=tf.optimizers.Adam(), 
              loss='binary_crossentropy',
              metrics=['accuracy'])




filepath = "modelCNN25thDec.h5"
#checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, 
#                            save_best_only=True, mode='max')

#reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=2, 
#                                   verbose=1, mode='max', min_lr=0.00001)
                              
                              
#callbacks_list = [checkpoint, reduce_lr]


history_main_tf = model.fit(
                    data_aug.flow_from_directory(test_dir,target_size=(224,224), batch_size=BATCH_SIZE, shuffle=True,
                         color_mode='grayscale'),
                    epochs = 20, 
                    validation_data=data_aug.flow_from_directory(val_dir,target_size=(224,224), batch_size=BATCH_SIZE, shuffle=True,
                         color_mode='grayscale')#,callbacks=callbacks_list
                    )



#2 Transfer learning with VGG16
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization


def build_advanced_net(model_weights=None, image_size: int = 224, classes: int = 2) -> Sequential:

    conv_base = VGG16(include_top=False,
                      weights='imagenet',
                      input_shape=(image_size, image_size, 3))
    for layer in conv_base.layers[:-2]:
        layer.trainable = False
    model = Sequential()
    model.add(conv_base)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(classes, activation='softmax'))
    if model_weights is not None:
        model.load_weights(model_weights)

    return model

#3 Transfer learning with vison transformer
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization


def build_advanced_net(model_weights=None, image_size: int = 224, classes: int = 2) -> Sequential:

    '''conv_base = VGG16(include_top=False,
                      weights='imagenet',
                      input_shape=(image_size, image_size, 3))'''
    
    vit_model = vit.vit_b32( image_size = image_size,
                                activation = 'softmax',
                                pretrained = True,
                                include_top = False,
                                pretrained_top = False,
                                classes=2 )
    for layer in vit_model.layers[:-2]:
        layer.trainable = False
    model = Sequential()
    model.add(vit_model)
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.BatchNormalization())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(classes, activation='softmax'))
    if model_weights is not None:
        model.load_weights(model_weights)

    return model

from vit_keras import visualize
from keras.preprocessing.image import load_img,img_to_array

vit_model = vit.vit_b32( image_size=224,
                                activation = 'softmax',
                                pretrained = True,
                                include_top = False,
                                pretrained_top = False,
                                classes = 2)

x = load_img('FinalYrProj\\Training\\Validation\\Not Stress\\Happy-10.jpg',target_size=(224,224))
image = img_to_array(x)
#image = image.reshape((1,image.shape[0],image.shape[1],image.shape[2]))


attention_map = visualize.attention_map(model = vit_model, image = image)

# Plot results
fig, (ax1, ax2) = plt.subplots(ncols = 2)
ax1.axis('off')
ax2.axis('off')
ax1.set_title('Original')
ax2.set_title('Attention Map')
_ = ax1.imshow(x)
_ = ax2.imshow(attention_map)

plt.show()


